In [1]:
from reasoning_engines.langchain_llm_wrappers import QuickAzureChatOpenAI
from reasoning_engines.langchain_llm_wrappers import QuickOpenAIClient
from reasoning_engines.langchain_llm_wrappers import QuickAzureOpenAIClient
openai_client = QuickOpenAIClient()
azure_openai_client = QuickAzureOpenAIClient('1106')
llm = QuickAzureChatOpenAI('1106')
# openai_response = openai_client.chat_completions_create(query='How is the weather in Seoul, Bangkok and LA?', tools=openai_functions)
# azure_openai_client.chat_completions_create(query='hello')
from tools.get_tools import DefaultSchemasTools
default_schemas_and_tools = DefaultSchemasTools(azure_gpt_version='1106')
schemas_and_tools = default_schemas_and_tools.schemas_and_tools()
schemas = default_schemas_and_tools.schemas()
tools = default_schemas_and_tools.tools()
tool_dictionary = default_schemas_and_tools.tool_dictionary()
from langchain.utils.openai_functions import convert_pydantic_to_openai_tool
openai_functions = [convert_pydantic_to_openai_tool(x) for x in schemas]


from utils.wrappers import retry
from openai import RateLimitError
@retry(allowed_exceptions=(RateLimitError,))
def get_function_response(function_name, function_args):
    return tool_dictionary[function_name].run(**function_args)


from agents.parallel_func_calling_agent import OpenAIParallelFuntionCallingAgent
from chains.qa_evaluators import CustomQAEvaluator

parallel_calling_agent = OpenAIParallelFuntionCallingAgent(
    reasoninig_engine = QuickAzureChatOpenAI('1106'),
    base_prompt = 'jet-taekyo-lee/parallel-function-calling-agent',
    schemas_and_tools = DefaultSchemasTools(azure_gpt_version='1106'),
    evaluator = CustomQAEvaluator(llm=QuickAzureChatOpenAI('1106')),
    action_word = 'Tool invocations in parallel',
    use_chat_completion_api = True,
    azure_apenai_client = QuickAzureOpenAIClient('1106'),
    horizon = 3
)
query="How old are the current Korean president and Joe Biden?"
# query="How old are the current Korean president?"
# query="What is the summation of the temperatures of Seoul, Tokyo, and New york?"
reference="Joe Biden is 81 years old and the current Korean president, Yoon Suk Yeol, is 62 years old."
parallel_calling_agent.clear_logs()


parallel_calling_agent.run_agent_trials(num_trials=1, 
query=query,
reference=reference)

import time
for _ in range(5):
    for s in range(60, 0, -1):
        print(s, end=' ')
        time.sleep(1)   
    parallel_calling_agent.clear_logs()

    parallel_calling_agent.agent_step(query)
    parallel_calling_agent.run_agent_trials(num_trials=3, 
    query=query,
    reference=reference)


[{'role': 'system',
  'content': 'You are a helpful assistant. Respond to the human as helpfully and accurately as possible in the same language as the human. But your intermediate processes should be done in English for more decent results. Answer in a consise manner with only a few words or a sentence if possible.'},
 {'role': 'user', 'content': ''}]